# Reducing QAOA Circuit Depth by Factoring out Semi-Symmetries

## References
1. https://arxiv.org/abs/2411.08824
2. https://arxiv.org/abs/2412.17841

In [1]:
import numpy as np
import networkx as nx
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import CplexOptimizer

Let us define QUBO matrix assuming that it is a upper triangular matrix. This is the same as "Table 1" in [1].

In [2]:
Q = np.array([
    [-1, 3, 0, 0, 3, 0],
    [0, -1, 3, 0, 3, 3],
    [0, 0, -1, 3, 3, 0],
    [0, 0, 0, -1, 0, 3],
    [0, 0, 0, 0, -1, 3],
    [0, 0, 0, 0, 0, -1],
])
Q.shape

(6, 6)

Plot the problem graph. (Please note that this graph is different from "Fig 1" in [1]. However, I think that this graph more correctly reflects the matrix.)

In [3]:
N, M = Q.shape
edges = []
for i in range(N):
    for j in range(i+1, M):
        if Q[i, j] > 0:
            edges.append((i+1, j+1))
G = nx.Graph(edges)
np.random.seed(0)
nx.draw(G, with_labels=True)

Implement `getConflictList` and `getMostSymQubits` functions.

In [4]:
def get_conflict_list(Q):
    cL = []
    n = Q.shape[0]
    Z = [sum([Q[i, j] for j in range(n) if Q[i, j] < 0])for i in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            if Q[i, j] > - Z[i] - Z[j]:
                cL.append((i, j))
    return cL


def get_most_symmetric_qubits(Q, cL):
    n = Q.shape[0]
    best = (0, 1)
    bestSyms = []
    for (i, j) in cL: # i < j in cL
        syms = [k for k in range(n) if Q[min(i, k), max(i, k)] == Q[min(j, k), max(j, k)] != 0]
        #print(i, j, len(syms))
        if len(syms) >= len(bestSyms):
            best = (i, j)
            bestSyms = syms
    return bestSyms, best

Check how many conflicted qubit pairs there are.

In [5]:
cL = get_conflict_list(Q)
cL

[(0, 1), (0, 4), (1, 2), (1, 4), (1, 5), (2, 3), (2, 4), (3, 5), (4, 5)]

Select the qubit pair that contains the most symmetric qubits.

In [6]:
bestSyms, best = get_most_symmetric_qubits(Q, cL)
bestSyms, best

([0, 2, 5], (1, 4))

Update QUBO matrix using `enhance` function where we reduce the coupling by adding new ancilla qubit.

`z` is a positive penalty value. If we choose $z = \sum_{(i, j)} |Q_{i,j}|$ at least, valid solutions $x$ have the same energy regarding $Q$ as to $Q_{\text{mod}}$ with best values for the ancilla qubits $x_{\text{mod}} = x + [x_a]$. Here, we set `z=3`.

In [7]:
def enhance(Q, i, j, syms, z):
    n = Q.shape[0]
    Q_mod = np.zeros((n+1, n+1))
    Q_mod[:n, :n] = np.triu(Q)
    Q_mod[i, i] = Q[i, i] + z
    Q_mod[j, j] = Q[j, j] + z
    Q_mod[n, n] = z
    Q_mod[i, n] = -2 * z
    Q_mod[j, n] = -2 * z
    Q_mod[i, j] = 2 * z
    for k in syms:
        Q_mod[k, n] = Q[min(i, k), max(i, k)]
        Q_mod[min(i, k), max(i, k)] = 0
        Q_mod[min(j, k), max(j, k)] = 0
    return Q_mod

In [8]:
Q_mod = enhance(Q, best[0], best[1], bestSyms, 3)

In [9]:
Q.shape

(6, 6)

In [10]:
Q_mod.shape

(7, 7)

In [11]:
Q

array([[-1,  3,  0,  0,  3,  0],
       [ 0, -1,  3,  0,  3,  3],
       [ 0,  0, -1,  3,  3,  0],
       [ 0,  0,  0, -1,  0,  3],
       [ 0,  0,  0,  0, -1,  3],
       [ 0,  0,  0,  0,  0, -1]])

In [12]:
Q_mod

array([[-1.,  0.,  0.,  0.,  0.,  0.,  3.],
       [ 0.,  2.,  0.,  0.,  6.,  0., -6.],
       [ 0.,  0., -1.,  3.,  0.,  0.,  3.],
       [ 0.,  0.,  0., -1.,  0.,  3.,  0.],
       [ 0.,  0.,  0.,  0.,  2.,  0., -6.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  3.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  3.]])

In [13]:
N = Q.shape[0]
density = np.count_nonzero(np.triu(Q)) / (N+N*(N-1)//2) * 100
print(f"Density (original): {density:.3f}%")

N_mod = Q_mod.shape[0]
density_mod = np.count_nonzero(np.triu(Q_mod)) / (N_mod+N_mod*(N_mod-1)//2) * 100
print(f"Density (mod): {density_mod:.3f}%")

Density (original): 71.429%
Density (mod): 53.571%


# Summary

In [14]:
def factoring_out_semi_symmetries(Q, num_ancillas=1, z=3):
    cL = get_conflict_list(Q)
    n = Q.shape[0]
    n_new = n
    while len(cL) > 0:
        syms, (i, j) = get_most_symmetric_qubits(Q, cL)
        if len(syms) < 3 or n_new == n + num_ancillas:
            break
        Q = enhance(Q, i, j, syms, z)
        n_new += 1
        cL = get_conflict_list(Q)
    return Q

In [15]:
Q_mod = factoring_out_semi_symmetries(Q)

In [16]:
Q_mod.shape

(7, 7)

In [17]:
np.triu(Q_mod)

array([[-1.,  0.,  0.,  0.,  0.,  0.,  3.],
       [ 0.,  2.,  0.,  0.,  6.,  0., -6.],
       [ 0.,  0., -1.,  3.,  0.,  0.,  3.],
       [ 0.,  0.,  0., -1.,  0.,  3.,  0.],
       [ 0.,  0.,  0.,  0.,  2.,  0., -6.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  3.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  3.]])

# Optimize QUBO

In [18]:
def convert_Q2op(Q):
    qubo = QuadraticProgram()
    n = Q.shape[0]
    for i in range(n):
        qubo.binary_var(f'x{i}')
    linear = {}
    quadratic = {}
    for i in range(n):
        for j in range(i, n):
            if i == j:
                linear[f"x{i}"] = Q[i, i]
            else:
                quadratic[f"x{i}", f"x{j}"] = Q[i, j]
    qubo.minimize(linear=linear, quadratic=quadratic)
    qubit_op, offset = qubo.to_ising()
    return qubo, qubit_op, offset

In [19]:
qubo, qubit_op, offset = convert_Q2op(Q)

In [20]:
cplex_result = CplexOptimizer().solve(qubo)
print(cplex_result.prettyprint())

objective function value: -3.0
variable values: x0=1.0, x1=0.0, x2=1.0, x3=0.0, x4=0.0, x5=1.0
status: SUCCESS


In [21]:
qubo_mod, qubit_op_mod, offset_mod = convert_Q2op(Q_mod)

In [22]:
cplex_result = CplexOptimizer().solve(qubo_mod)
print(cplex_result.prettyprint())

objective function value: -3.0
variable values: x0=1.0, x1=0.0, x2=1.0, x3=0.0, x4=0.0, x5=1.0, x6=0.0
status: SUCCESS
